In [0]:
#librerias
##### Para cargar en Jupyter #####
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from skimage import io
from tqdm import tqdm

# Text processing
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split

# Text 
from keras.utils import np_utils, plot_model
from keras.layers import Embedding, LSTM, Reshape, concatenate
from keras.optimizers import RMSprop
from keras.callbacks import History
#from pydot import pydot

## Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2
from keras.layers import Conv2D, RepeatVector, TimeDistributed, Activation, GlobalMaxPooling2D, MaxPooling2D, Flatten, Dropout, Dense, Input, BatchNormalization#, merge, Merge
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, Sequential

%matplotlib inline

In [0]:
#path = '/Users/sabygalvan/Desktop/FinalDL/'
path ="C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject"
dataTraining = pd.read_csv(os.path.join(path, 'dataTraining.csv'), encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv(os.path.join(path, 'dataTesting.csv'), encoding='UTF-8', index_col=0)

In [0]:
# Funcion para seleccionar una muestra de las imagenes
def sampleData(dataTrain, dataTest, varTrain, varTest):
    ix_train=np.random.choice(dataTrain.shape[0], varTrain)
    ix_test=np.random.choice(dataTest.shape[0], varTest)
    return dataTrain.iloc[ix_train], dataTest.iloc[ix_test], ix_train, ix_test

newDataTraining, newDataTesting, ix_train, ix_test  = sampleData(dataTraining, dataTesting, varTrain=500, varTest=50)

In [0]:
path ="C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject/images_original"
#path='images/'
# Training
images_training = []
newIndexTrain = []
for i in tqdm(newDataTraining.index):
    #path ="C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject/"
    #images_training.append(io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg')).flatten())
    #images_training.append(io.imread(os.path.join(path, str(i) + '_resize_gray.jpeg')))
    archivoImg = cv2.resize(io.imread(os.path.join(path, str(i) + ".jpeg")), (224, 224))
    if len(archivoImg.shape) == 3:
        images_training.append(archivoImg)
        newIndexTrain.append(i)
images_training = np.stack(images_training)
images_training.shape, dataTraining.shape[0], newDataTraining.shape

100%|██████████| 500/500 [00:13<00:00, 36.67it/s]


((498, 224, 224, 3), 7895, (500, 5))

In [0]:
#newDataTraining = dataTraining.loc[newIndexTrain]
#newDataTraining['genres'] = newDataTraining['genres'].map(lambda x: eval(str(x)))
#le = MultiLabelBinarizer()
#y_genres = le.fit_transform(newDataTraining['genres'])
#y_genres.shape

In [0]:
#vect = CountVectorizer(analyzer=split_into_lemmas, ngram_range=(1, 2), max_features=1000, stop_words='english')
#vect = CountVectorizer(analyzer="word", ngram_range=(1, 2), max_features=100, stop_words='english')
#X_dtm = vect.fit_transform(newDataTraining['plot'])
#X_dtm.shape

In [0]:
# extract features from each photo in the directory
def extract_features(directory):
    from keras.applications.vgg16 import preprocess_input
    # load the model
    in_layer = Input(shape=(224, 224, 3))
    model = VGG16(include_top=False, input_tensor=in_layer)
    print(model.summary())
    # extract features from each photo
    features = dict()
    for name in tqdm(os.listdir(directory)):
        # load an image from file
        filename = directory + '/' + name
        image = load_img(filename, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the VGG model
        image = preprocess_input(image)
        # get features
        feature = model.predict(image, verbose=0)
        # get image id
        image_id = name.split('.')[0]
        # store feature
        features[image_id] = feature
        print('>%s' % name)
    return features

In [0]:
directory="C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject/images_original"
features = extract_features(directory)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

C:\Users\da.salazarb\AppData\Local\Continuum\anaconda3\envs\PythonR\lib\site-packages\PIL\Image.py:2509: DecompressionBombWarning: Image size (97200000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


>119.jpeg
>1190.jpeg
>1191.jpeg
>1192.jpeg
>1193.jpeg
>1194.jpeg
>1195.jpeg
>1196.jpeg
>1197.jpeg
>1198.jpeg
>1199.jpeg
>12.jpeg
>120.jpeg
>1200.jpeg
>1201.jpeg
>1202.jpeg
>1203.jpeg
>1204.jpeg
>1205.jpeg
>1206.jpeg
>1207.jpeg
>1208.jpeg
>1209.jpeg
>121.jpeg
>1210.jpeg
>1211.jpeg
>1212.jpeg
>1213.jpeg
>1214.jpeg
>1215.jpeg
>1216.jpeg
>1217.jpeg
>1218.jpeg
>1219.jpeg
>122.jpeg
>1220.jpeg
>1221.jpeg
>1222.jpeg
>1223.jpeg
>1224.jpeg
>1225.jpeg
>1226.jpeg
>1227.jpeg
>1228.jpeg
>1229.jpeg
>123.jpeg
>1230.jpeg
>1231.jpeg
>1232.jpeg
>1233.jpeg
>1234.jpeg
>1235.jpeg
>1236.jpeg
>1237.jpeg
>1238.jpeg
>1239.jpeg
>124.jpeg
>1240.jpeg
>1241.jpeg
>1242.jpeg
>1243.jpeg
>1244.jpeg
>1245.jpeg
>1246.jpeg
>1247.jpeg
>1248.jpeg
>1249.jpeg
>125.jpeg
>1250.jpeg
>1251.jpeg
>1252.jpeg
>1253.jpeg
>1254.jpeg
>1255.jpeg
>1256.jpeg
>1257.jpeg
>1258.jpeg
>1259.jpeg
>126.jpeg
>1260.jpeg
>1261.jpeg
>1262.jpeg
>1263.jpeg
>1264.jpeg
>1265.jpeg
>1266.jpeg
>1267.jpeg
>1268.jpeg
>1269.jpeg
>127.jpeg
>1270.jpeg
>1271.jpeg

C:\Users\da.salazarb\AppData\Local\Continuum\anaconda3\envs\PythonR\lib\site-packages\PIL\Image.py:2509: DecompressionBombWarning: Image size (96592500 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


>1703.jpeg
>1704.jpeg
>1705.jpeg
>1706.jpeg
>1707.jpeg
>1708.jpeg
>1709.jpeg
>171.jpeg
>1710.jpeg
>1711.jpeg
>1712.jpeg
>1713.jpeg
>1714.jpeg
>1715.jpeg
>1716.jpeg
>1717.jpeg
>1718.jpeg
>1719.jpeg
>172.jpeg
>1720.jpeg
>1721.jpeg
>1722.jpeg
>1723.jpeg
>1724.jpeg
>1725.jpeg
>1726.jpeg
>1727.jpeg
>1728.jpeg
>1729.jpeg
>173.jpeg
>1730.jpeg
>1731.jpeg
>1732.jpeg
>1733.jpeg
>1734.jpeg
>1735.jpeg
>1736.jpeg
>1737.jpeg
>1738.jpeg
>1739.jpeg
>174.jpeg
>1740.jpeg
>1741.jpeg
>1742.jpeg
>1743.jpeg
>1744.jpeg
>1745.jpeg
>1746.jpeg
>1747.jpeg
>1748.jpeg
>1749.jpeg
>175.jpeg
>1750.jpeg
>1751.jpeg
>1752.jpeg
>1753.jpeg
>1754.jpeg
>1755.jpeg
>1756.jpeg
>1757.jpeg
>1758.jpeg
>1759.jpeg
>176.jpeg
>1760.jpeg
>1761.jpeg
>1762.jpeg
>1763.jpeg
>1764.jpeg
>1765.jpeg
>1766.jpeg
>1767.jpeg
>1768.jpeg
>1769.jpeg
>177.jpeg
>1770.jpeg
>1771.jpeg
>1772.jpeg
>1773.jpeg
>1774.jpeg
>1775.jpeg
>1776.jpeg
>1777.jpeg
>1778.jpeg
>1779.jpeg
>178.jpeg
>1780.jpeg
>1781.jpeg
>1782.jpeg
>1783.jpeg
>1784.jpeg
>1785.jpeg
>1786.j

...

>4483.jpeg
>4484.jpeg
>4485.jpeg
>4486.jpeg
>4487.jpeg
>4488.jpeg
>4489.jpeg
>449.jpeg
>4490.jpeg
>4491.jpeg
>4492.jpeg
>4493.jpeg
>4494.jpeg
>4495.jpeg
>4496.jpeg
>4497.jpeg
>4498.jpeg
>4499.jpeg
>45.jpeg
>450.jpeg
>4500.jpeg
>4501.jpeg
>4502.jpeg
>4503.jpeg
>4504.jpeg
>4505.jpeg
>4506.jpeg
>4507.jpeg
>4508.jpeg
>4509.jpeg
>451.jpeg
>4510.jpeg
>4511.jpeg
>4512.jpeg
>4513.jpeg
>4514.jpeg
>4515.jpeg
>4516.jpeg
>4517.jpeg
>4518.jpeg
>4519.jpeg
>452.jpeg
>4520.jpeg
>4521.jpeg
>4522.jpeg
>4523.jpeg
>4524.jpeg
>4525.jpeg
>4526.jpeg
>4527.jpeg
>4528.jpeg
>4529.jpeg
>453.jpeg
>4530.jpeg
>4531.jpeg
>4532.jpeg
>4533.jpeg
>4534.jpeg
>4535.jpeg
>4536.jpeg
>4537.jpeg
>4538.jpeg
>4539.jpeg
>454.jpeg
>4540.jpeg
>4541.jpeg
>4542.jpeg
>4543.jpeg
>4544.jpeg
>4545.jpeg
>4546.jpeg
>4547.jpeg
>4548.jpeg
>4549.jpeg
>455.jpeg
>4550.jpeg
>4551.jpeg
>4552.jpeg
>4553.jpeg
>4554.jpeg
>4555.jpeg
>4556.jpeg
>4557.jpeg
>4558.jpeg
>4559.jpeg
>456.jpeg
>4560.jpeg
>4561.jpeg
>4562.jpeg
>4563.jpeg
>4564.jpeg
>4565.jpe

C:\Users\da.salazarb\AppData\Local\Continuum\anaconda3\envs\PythonR\lib\site-packages\PIL\Image.py:2509: DecompressionBombWarning: Image size (94770000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


>8274.jpeg
>8275.jpeg
>8276.jpeg
>8277.jpeg
>8278.jpeg
>8279.jpeg
>828.jpeg
>8280.jpeg
>8281.jpeg
>8282.jpeg
>8283.jpeg
>8284.jpeg
>8285.jpeg
>8286.jpeg
>8287.jpeg
>8288.jpeg
>8289.jpeg
>829.jpeg
>8290.jpeg
>8291.jpeg
>8292.jpeg
>8293.jpeg
>8294.jpeg
>8295.jpeg
>8296.jpeg
>8297.jpeg
>8298.jpeg
>8299.jpeg
>83.jpeg
>830.jpeg
>8300.jpeg
>8301.jpeg
>8302.jpeg
>8303.jpeg
>8304.jpeg
>8305.jpeg
>8306.jpeg
>8307.jpeg
>8308.jpeg
>8309.jpeg
>831.jpeg
>8310.jpeg
>8311.jpeg
>8312.jpeg
>8313.jpeg
>8314.jpeg
>8315.jpeg
>8316.jpeg
>8317.jpeg
>8318.jpeg
>8319.jpeg
>832.jpeg
>8320.jpeg
>8321.jpeg
>8322.jpeg
>8323.jpeg
>8324.jpeg
>8325.jpeg
>8326.jpeg
>8327.jpeg
>8328.jpeg
>8329.jpeg
>833.jpeg
>8330.jpeg
>8331.jpeg
>8332.jpeg
>8333.jpeg
>8334.jpeg
>8335.jpeg
>8336.jpeg
>8337.jpeg
>8338.jpeg
>8339.jpeg
>834.jpeg
>8340.jpeg
>8341.jpeg
>8342.jpeg
>8343.jpeg
>8344.jpeg
>8345.jpeg
>8346.jpeg
>8347.jpeg
>8348.jpeg
>8349.jpeg
>835.jpeg
>8350.jpeg
>8351.jpeg
>8352.jpeg
>8353.jpeg
>8354.jpeg
>8355.jpeg
>8356.jpe

C:\Users\da.salazarb\AppData\Local\Continuum\anaconda3\envs\PythonR\lib\site-packages\PIL\Image.py:2509: DecompressionBombWarning: Image size (92984898 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


>9276.jpeg
>9277.jpeg
>9278.jpeg
>9279.jpeg
>928.jpeg
>9280.jpeg
>9281.jpeg
>9282.jpeg
>9283.jpeg
>9284.jpeg
>9285.jpeg
>9286.jpeg
>9287.jpeg
>9288.jpeg
>9289.jpeg
>929.jpeg
>9290.jpeg
>9291.jpeg
>9292.jpeg
>9293.jpeg
>9294.jpeg
>9295.jpeg
>9296.jpeg
>9297.jpeg
>9298.jpeg
>9299.jpeg
>93.jpeg
>930.jpeg
>9300.jpeg
>9301.jpeg
>9302.jpeg
>9303.jpeg
>9304.jpeg
>9305.jpeg
>9306.jpeg
>9307.jpeg
>9308.jpeg
>9309.jpeg
>931.jpeg
>9310.jpeg
>9311.jpeg
>9312.jpeg
>9313.jpeg
>9314.jpeg
>9315.jpeg
>9316.jpeg
>9317.jpeg
>9318.jpeg
>9319.jpeg
>932.jpeg
>9320.jpeg
>9321.jpeg
>9322.jpeg
>9323.jpeg
>9324.jpeg
>9325.jpeg
>9326.jpeg
>9327.jpeg
>9328.jpeg
>9329.jpeg
>933.jpeg
>9330.jpeg
>9331.jpeg
>9332.jpeg
>9333.jpeg
>9334.jpeg
>9335.jpeg
>9336.jpeg
>9337.jpeg
>9338.jpeg
>9339.jpeg
>934.jpeg
>9340.jpeg
>9341.jpeg
>9342.jpeg
>9343.jpeg
>9344.jpeg
>9345.jpeg
>9346.jpeg
>9347.jpeg
>9348.jpeg
>9349.jpeg
>935.jpeg
>9350.jpeg
>9351.jpeg
>9352.jpeg
>9353.jpeg
>9354.jpeg
>9355.jpeg
>9356.jpeg
>9357.jpeg
>9358.jpe

In [0]:
from pickle import dump

In [0]:
# save to file
dump(features, open('features.pkl', 'wb'))

In [0]:
# load photo features
def load_photo_features(filename, dataset):
    from pickle import load
    # load all features
    all_features = load(open(filename, 'rb'))
    # filter features
    features = {k: all_features[str(k)] for k in dataset}
    return features

In [0]:
train_features = load_photo_features('features.pkl', ix_train)
len(train_features)

388

In [0]:
train_features_list = []
ix_train_features = []
for key in train_features:
    train_features_list.append(train_features[key])
    ix_train_features.append(key)

In [0]:
len(ix_train_features)

388

In [0]:
train_features_array = np.vstack(train_features_list)
train_features_array = train_features_array.astype('float32')
train_features_array /= 255

In [0]:
# Funcion para seleccionar los plot de las peliculas
def sampleData(dataTraining, newIndexTrain):
    #ix_train=np.random.choice(dataTrain.shape[0], varTrain)
    #ix_test=np.random.choice(dataTest.shape[0], varTest)
    newDataTraining = dataTraining.iloc[newIndexTrain]
    newDataTraining['genres'] = newDataTraining['genres'].map(lambda x: eval(str(x)))
    le = MultiLabelBinarizer()
    y_genres = le.fit_transform(newDataTraining['genres'])
    #y_genres.shape
    #vect = CountVectorizer(analyzer=split_into_lemmas, ngram_range=(1, 2), max_features=1000, stop_words='english')
    vect = CountVectorizer(analyzer="word", ngram_range=(1, 2), max_features=max_review_length, stop_words='english')
    X_dtm = vect.fit_transform(newDataTraining['plot'])
    return y_genres, X_dtm

y_genres, X_dtm = sampleData(dataTraining, ix_train_features)
y_genres.shape, X_dtm.shape

C:\Users\da.salazarb\AppData\Local\Continuum\anaconda3\envs\PythonR\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


((388, 24), (388, 1000))

In [0]:
max_review_length = 1000
top_words = 1000

In [0]:
def define_model(vocab_size, max_length):
    # feature extractor (encoder)
    inputs1 = Input(shape=(7, 7, 512))
    fe1 = GlobalMaxPooling2D()(inputs1)
    fe2 = Dense(128, activation='tanh')(fe1)
    fe3 = RepeatVector(max_length)(fe2)
    # embedding
    inputs2 = Input(shape=(max_length,))
    emb2 = Embedding(vocab_size, 50, mask_zero=True)(inputs2)
    emb3 = LSTM(256, return_sequences=True)(emb2)
    emb4 = TimeDistributed(Dense(128, activation='tanh'))(emb3)
    # merge inputs
    merged = concatenate([fe3, emb4])
    # language model (decoder)
    #lm2 = Dense(500, activation='relu')(merged)
    lm2 = LSTM(500)(merged)
    #lm2a = Dropout(0.5)(lm2)
    #lm2a = BatchNormalization()(lm2)
    lm3 = Dense(500, activation='tanh')(lm2)
    outputs = Dense(24, activation='sigmoid')(lm3)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    print(model.summary())
    #plot_model(model, show_shapes=True, to_file='plot.png')
    return model

In [0]:
max_review_length = 1000
top_words = 1000
model = define_model(vocab_size=top_words, max_length=max_review_length)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           (None, 7, 7, 512)    0                                            
__________________________________________________________________________________________________
input_45 (InputLayer)           (None, 1000)         0                                            
__________________________________________________________________________________________________
global_max_pooling2d_21 (Global (None, 512)          0           input_44[0][0]                   
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 1000, 50)     50000       input_45[0][0]                   
__________________________________________________________________________________________________
dense_80 (

In [0]:
train_features_array.shape, X_dtm.shape, y_genres.shape, ix_train.shape

((388, 7, 7, 512), (388, 1000), (388, 24), (400,))

In [0]:
model.fit([train_features_array, X_dtm], y_genres, epochs=3, batch_size=50)

In [0]:
test_features = load_photo_features('features.pkl', dataTesting.index)
len(test_features), dataTesting.index.shape

(3383, (3383,))

In [0]:
test_features_list = []
ix_test_features = []
for key in test_features:
    test_features_list.append(test_features[key])
    ix_test_features.append(key)
test_features_array = np.vstack(test_features_list)
test_features_array = test_features_array.astype('float32')
test_features_array /= 255
test_features_array.shape

(3383, 7, 7, 512)

In [0]:
vect = CountVectorizer(analyzer="word", ngram_range=(1, 2), max_features=1000, stop_words='english')
X_dtm_test = vect.fit_transform(dataTesting['plot'])
X_dtm_test.shape

(3383, 1000)

In [0]:
cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

y_pred_test_genres = model.predict([test_features_array, X_dtm_test])
#y_pred_test_genres = model.predict_proba(images_testing)

pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols).to_csv('pred_genres_img_RF.csv', index_label='ID')

In [0]:
os.getcwd()

'C:\\Users\\da.salazarb\\Google Drive (dasalazarb@unal.edu.co)\\AppliedDeepLearningClass-master\\finalProject'

In [0]:
## Texto

# Funcion para seleccionar una muestra de las imagenes
def sampleData(dataTrain, dataTest, varTrain, varTest):
    ix_train=np.random.choice(dataTrain.shape[0], varTrain)
    ix_test=np.random.choice(dataTest.shape[0], varTest)
    return dataTrain.iloc[ix_train], dataTest.iloc[ix_test]

newDataTraining, newDataTesting  = sampleData(dataTraining, dataTesting, varTrain=1000, varTest=50)

In [0]:
# PCA imagenes

#path ="C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject/images_original"
#path='images/'
# Training
images_training = []
newIndexTrain = []
for i in tqdm(newDataTraining.index):
    path ="C:/Users/da.salazarb/Google Drive (dasalazarb@unal.edu.co)/AppliedDeepLearningClass-master/finalProject/"
    imagenes = io.imread(os.path.join(path, 'images_resize_gray', str(i) + '_resize_gray.jpeg'))
    #imagenes.astype('float32')
    imagenes = imagenes / 255
    images_training.append(imagenes.flatten())
    newIndexTrain.append(i)
    #images_training.append(io.imread(os.path.join(path, str(i) + '_resize_gray.jpeg')))
    #archivoImg = cv2.resize(io.imread(os.path.join(path, str(i) + ".jpeg")), (224, 224))
    #if len(archivoImg.shape) == 3:
        #images_training.append(archivoImg)
        #newIndexTrain.append(i)
images_training = np.stack(images_training)
images_training.shape, dataTraining.shape[0], newDataTraining.shape

newDataTraining = dataTraining.loc[newIndexTrain]
newDataTraining['genres'] = newDataTraining['genres'].map(lambda x: eval(str(x)))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(newDataTraining['genres'])



max_review_length = 500
vect = CountVectorizer(analyzer="word", ngram_range=(1, 2), max_features=max_review_length, stop_words='english')
X_dtm = vect.fit_transform(newDataTraining['plot'])

y_genres.shape, X_dtm.shape

100%|██████████| 1000/1000 [00:01<00:00, 798.00it/s]
C:\Users\da.salazarb\AppData\Local\Continuum\anaconda3\envs\PythonR\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


((1000, 24), (1000, 500))

In [0]:
pca = PCA(n_components=500)
images_training_pca = pca.fit_transform(images_training)
images_training_pca.shape

(1000, 500)

In [0]:
import keras

input_text = Input(shape=(max_review_length,))
Text = Embedding(top_words, 32)(input_text)
Text = LSTM(256)(Text)
Text = BatchNormalization()(Text)
Text = Dropout(0.5)(Text)
Text = Dense(256, activation='tanh')(Text)

input_Img = Input(shape=(max_review_length, )) # (224,224, 3)
Img = Dense(256)(input_Img)
Img = Activation("tanh")(Img)
Img = Dropout(0.5)(Img)
Img = BatchNormalization(axis=1)(Img)

modelTextImg = keras.layers.concatenate([Text, Img])

#modelTextImg = Flatten()(modelTextImg)
modelTextImg = Dense(200, activation='relu')(modelTextImg)
modelTextImg = Dropout(0.5)(modelTextImg)
modelTextImg = Dense(24, activation='sigmoid')(modelTextImg)

modelTextImg = Model(inputs=[input_text, input_Img], outputs=modelTextImg)
modelTextImg.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [0]:
modelTextImg.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_26 (InputLayer)           (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, 1000, 64)     64000       input_26[0][0]                   
__________________________________________________________________________________________________
input_27 (InputLayer)           (None, 1000)         0                                            
__________________________________________________________________________________________________
lstm_14 (LSTM)                  (None, 256)          328704      embedding_14[0][0]               
__________________________________________________________________________________________________
dense_30 (

In [0]:
images_training_pca.shape, X_dtm_test.shape

((1000, 500), (1000, 500))

In [0]:
modelTextImg.fit([X_dtm_test, images_training_pca], y_genres, epochs = 15, batch_size=100)

Epoch 1/15
1000/1000 [==============================] - 27s 27ms/step - loss: 7.4252 - acc: 0.1910
Epoch 2/15
1000/1000 [==============================] - 28s 28ms/step - loss: 7.2812 - acc: 0.2090
Epoch 3/15
1000/1000 [==============================] - 29s 29ms/step - loss: 7.1631 - acc: 0.2310
Epoch 4/15
1000/1000 [==============================] - 30s 30ms/step - loss: 7.0435 - acc: 0.2360
Epoch 5/15
1000/1000 [==============================] - 30s 30ms/step - loss: 6.9258 - acc: 0.2240
Epoch 6/15
1000/1000 [==============================] - 31s 31ms/step - loss: 6.8348 - acc: 0.2270
Epoch 7/15
1000/1000 [==============================] - 31s 31ms/step - loss: 6.7380 - acc: 0.2470
Epoch 8/15
1000/1000 [==============================] - 32s 32ms/step - loss: 6.7138 - acc: 0.2540
Epoch 9/15
1000/1000 [==============================] - 32s 32ms/step - loss: 6.5815 - acc: 0.2210
Epoch 10/15
1000/1000 [==============================] - 33s 33ms/step - loss: 6.4561 - acc: 0.2320
Epoch 11/